In [ ]:
# Imports
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

plt.rcParams['figure.figsize'] = (15.0, 8.0) # set default size of plots
plt.rcParams['figure.facecolor'] = 'white'

pd.set_option('display.max_rows', None)

matplotlib.rcParams.update({'font.size': 15})

In [ ]:
test_names = ['2mm', '3mm', 'adi', 'atax', 'bicg', 'cholesky', 'correlation', 'covariance', 'deriche', 'doitgen', 'durbin', 'fdtd-2d', 'floyd-warshall', 'gemm', 'gemver', 'gesummv', 'gramschmidt', 'heat-3d', 'jacobi-1d', 'jacobi-2d', 'lu', 'ludcmp', 'mvt', 'nussinov', 'seidel-2d', 'symm', 'syr2k', 'syrk', 'trisolv', 'trmm']

native_data = pd.DataFrame()
wasm_data = pd.DataFrame()
wasm_sgx_data = pd.DataFrame()

for test_name in test_names:
    native_test_data = pd.read_csv("benchmark-native-" + test_name + ".csv", names = ["time"])
    wasm_test_data = pd.read_csv("benchmark-wasm-" + test_name + ".csv", names = ["time"])
    wasm_sgx_test_data = pd.read_csv("benchmark-wasm-sgx-" + test_name + ".csv", names = ["time"])
    
    native_data[test_name] = native_test_data["time"]
    wasm_data[test_name] = wasm_test_data["time"]
    wasm_sgx_data[test_name] = wasm_sgx_test_data["time"]

In [ ]:
# Normalize the results based on native memory implementation
for col in wasm_data:
    wasm_sgx_data[col] = wasm_sgx_data[col] / native_data[col]
    wasm_data[col] = wasm_data[col] / native_data[col]
    native_data[col] = 1

In [ ]:
x = np.arange(len(test_names))  # the label locations
width = 0.7/2.0  # the width of the bars

fig, ax = plt.subplots()
native_bar = ax.bar(x - width, native_data.median().values.tolist(), width, label='Native')
wasm_bar = ax.bar(x, wasm_data.median().values.tolist(), width, label='WebAssembly AoT')
wasm_sgx_bar = ax.bar(x + width, wasm_sgx_data.median().values.tolist(), width, label='WebAssembly AoT in SGX')
        
# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_xlabel('Name of experiment')
ax.set_ylabel('Normalised runtime\n(Native = 1)')
ax.set_title('Polybench benchmark')
ax.set_xticks(x)
ax.set_xticklabels(test_names)
plt.xticks(rotation=90)
ax.legend()


handles, labels = ax.get_legend_handles_labels()
ax.legend(reversed(handles), reversed(labels), loc='upper left')  # reverse to keep order consistent

fig.tight_layout()

plt.show()

In [ ]:
# Export script
#
# Files generated:
# - polybench_wasm_formatted.csv
# - polybench_wasm_sgx_formatted.csv
#
# File format: experiment_name, mean, stddev

def export_to_file(dataset, filename):
    file = pd.DataFrame(columns = ["experiment_name", "mean", "stddev"])
    
    i = 0
    for test_name in test_names:
        file.loc[i] = [test_name,dataset[test_name].median(), dataset[test_name].std()] #
        i += 1
        
    display(file)
    file.to_csv(filename, index=False)
    
export_to_file(wasm_data, 'polybench_wasm_formatted.csv')
export_to_file(wasm_sgx_data, 'polybench_wasm_sgx_formatted.csv')